In [3]:
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.append(os.path.join(os.path.abspath(''), '../'))

from models.graph_sage_model import GraphSAGE
from pprint import pprint
from dataset.data_loader import DataLoader, playtime_forever_users_games_edge_scoring_function

import networkx as nx
from torch_geometric.data import Data
import torch
from torch_geometric.nn import SAGEConv
import pickle
from utils.utils import get_game_name_and_scores

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


c:\Users\Jeffrey\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
%reload_ext autoreload

In [5]:
data_loader = DataLoader(
    users_games_edge_scoring_function=playtime_forever_users_games_edge_scoring_function,
    user_game_edge_embeddings=['playtime_forever'],
    cache_local_dataset=True)
data_loader.load_random_edge_train_test_split(train_percentage=0.9, test_percentage=0.1, seed=0)


In [ ]:
# data_loader.save_data_loader_parameters('test_graphsage_default_data_loader', overwrite=True)
print(data_loader.users_games_df)

In [ ]:
# # Making NetworkX Graph

# G = nx.Graph()
# for index, row in data_loader.users_games_df.iterrows():
#     G.add_node(row['user_id'], node_type='user')
#     G.add_node(row['game_id'], node_type='game')
#     G.add_edge(row['user_id'], row['game_id'], playtime=row['playtime_forever'])

In [ ]:
# import pickle
# nx.write_gpickle(G, '/content/drive/MyDrive/my_graph.gpickle')

# with open('my_graph.pkl', 'wb') as f:
#     pickle.dump(G, f)

In [6]:
with open('my_graph.pkl', 'rb') as f:
    loaded_G = pickle.load(f)

# print(nx.is_isomorphic(G, loaded_G))
loaded_G

In [ ]:
len(list(loaded_G.edges))

In [7]:
edges = list(loaded_G.edges())[:1000]  
user_ids = list(set([edge[0] for edge in edges]))
game_ids = list(set([edge[1] for edge in edges]))

In [ ]:
# node_ids = list(set(user_ids + game_ids))
# print("Total unique nodes:", len(node_ids))
# print("User IDs in node_ids:", all(user_id in node_ids for user_id in user_ids))


In [8]:
# Combine user and game IDs into a single list of unique node IDs
node_ids = list(set(user_ids + game_ids))

# Create the edge index data using the combined node IDs
edge_index_data = [(node_ids.index(edge[0]), node_ids.index(edge[1])) for edge in edges]

# Convert the edge index list to a tensor
edge_index = torch.tensor(edge_index_data, dtype=torch.long).t().contiguous()

# Create the feature matrix x with a feature for each node in the combined list
x = torch.arange(len(node_ids), dtype=torch.float).view(-1, 1)

# Create the PyTorch Geometric Data object
data = Data(x=x, edge_index=edge_index)



In [ ]:
user_ids

In [ ]:
# edge_index = torch.tensor(data_loader.users_games_df[['user_id', 'game_id']].values.T, dtype=torch.long)
# num_nodes = max(data_loader.users_games_df['user_id'].max(), data_loader.users_games_df['game_id'].max()) + 1
# num_features = 10 
# x = torch.rand((num_nodes, num_features), dtype=torch.float)

# data = Data(x=x, edge_index=edge_index)


In [15]:
model = GraphSAGE(
    in_channels=data.x.size(1),
    hidden_channels=[32, 64],  
    out_channels=16,
    num_epochs=20,
    learning_rate=0.01,
    regularization=0.05,
)

# user_ids = data_loader.users_games_df['user_id'].unique().tolist()
# game_ids = data_loader.users_games_df['game_id'].unique().tolist()

model.train(data, user_ids, game_ids, debug=True)

data type: <class 'torch_geometric.data.data.Data'>
Has attribute 'x': True
Has attribute 'edge_index': True
Input x shape: torch.Size([1001, 1])
Input edge_index shape: torch.Size([2, 1000])
Shape after layer 0: torch.Size([1001, 32])
Shape after layer 1: torch.Size([1001, 64])
Output shape: torch.Size([1001, 16])
Epoch 1, Loss: 2896.034912109375
Input x shape: torch.Size([1001, 1])
Input edge_index shape: torch.Size([2, 1000])
Shape after layer 0: torch.Size([1001, 32])
Shape after layer 1: torch.Size([1001, 64])
Output shape: torch.Size([1001, 16])
Epoch 2, Loss: 1045.2039794921875
Input x shape: torch.Size([1001, 1])
Input edge_index shape: torch.Size([2, 1000])
Shape after layer 0: torch.Size([1001, 32])
Shape after layer 1: torch.Size([1001, 64])
Output shape: torch.Size([1001, 16])
Epoch 3, Loss: 981.011962890625
Input x shape: torch.Size([1001, 1])
Input edge_index shape: torch.Size([2, 1000])
Shape after layer 0: torch.Size([1001, 32])
Shape after layer 1: torch.Size([1001, 64

In [ ]:
print(data)

In [ ]:
# Assuming 'data' is your PyTorch Geometric Data object
score = model.get_score_between_user_and_game(730, 76561199045179546, data)
print(score)

In [16]:
top_10_games = model.score_and_predict_n_games_for_user(76561198835352289, N=10)
print(top_10_games)


Input x shape: torch.Size([1001, 1])
Input edge_index shape: torch.Size([2, 1000])
Shape after layer 0: torch.Size([1001, 32])
Shape after layer 1: torch.Size([1001, 64])
Output shape: torch.Size([1001, 16])
[(1172470, 10.265390396118164), (76561199197087868, 6.271185874938965), (76561199040442492, 6.239022731781006), (76561199508379772, 6.214553356170654), (76561198873487488, 6.181817531585693), (76561198368774268, 6.157919883728027), (76561199030186110, 6.124604225158691), (76561198172729467, 6.101285934448242), (76561198371721346, 6.067393779754639), (76561198805241978, 6.044651508331299)]
